In [1]:
import torch;import torch.nn as nn;import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from layer.modules.ssd import SSD
from layer.function.iou import match
a = torch.rand(2,3,300,300)
b = SSD()
c = b(a) # (loc, conf, defult)
d = torch.tensor([[0.0,0.0,1.0,1.0,2.0],[0.5,0.5,1.0,1.0,5.0]]).reshape(2,-1) #fake truthbox
d1 = torch.tensor((0.0,0.0,1.0,1.0,2.0)).reshape(1,-1)

In [2]:
matchbox = torch.cuda.FloatTensor(*c[1].shape)
loc_conf = torch.cuda.FloatTensor(*c[0].shape)

for i,j in enumerate((d,d1)):
    e = match(c[2],j)
    matchbox[i].zero_().scatter_(1,e[1].unsqueeze(1),1.0)
    loc_conf[i] = e[0]

In [3]:
print(matchbox.shape,loc_conf.shape,c[0].shape)

torch.Size([2, 8732, 21]) torch.Size([2, 8732, 4]) torch.Size([2, 8732, 4])


In [4]:
pos = matchbox[:,:,-1] == 0

pos_idx = pos.unsqueeze(pos.dim()).expand_as(c[0])
loc_p = c[0].masked_select(pos_idx).view(-1,4)
loc_t = loc_conf.masked_select(pos_idx).view(-1,4)
loss_l = F.smooth_l1_loss(loc_p,loc_t,reduction='sum')

In [5]:
loss_l

tensor(7.1530, grad_fn=<SmoothL1LossBackward>)

In [ ]:
loss_c = c[1].clone()
pos = matchbox == 1
loss_c[pos] = 0

In [ ]:
_, loss_idx = loss_c.sort(1,descending=True)
_, idx_rank = loss_idx.sort(1)
neg = idx_rank < num_neg.expand_as(idx_rank)

In [ ]:
indexbase = c[0][pos_idx].view(-1,4)
maskedselectbase = c[0].masked_select(pos_idx).view(-1,4)
same = indexbase != maskedselectbase
print(same.any()) # if indexbase and maskedselectbase is all same, return false(0.0)

In [34]:
c[0].masked_select(pos_idx).shape

torch.Size([616])

```python
batch_conf = conf_data.view(-1, self.num_classes) # conf_data : c[1], shape [ N, num_defaultbox, num_classes ]
# batch_conf : shape [ N * num_defaultbox, num_classes ]
loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1)) # loss_c = shape [ N * num_defaultbox ]
```

In [6]:
print(c[1].shape, matchbox.shape)

torch.Size([2, 8732, 21]) torch.Size([2, 8732, 21])


In [7]:
def log_sum_exp(x):
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x-x_max), 1, keepdim=True)) + x_max

In [8]:
tmp = torch.rand(5,5)
print(tmp.shape, log_sum_exp(tmp).shape)

torch.Size([5, 5]) torch.Size([5, 1])
